In [22]:
import os
import time
import sys
import warnings

sys.path.append("/home/prakank/anaconda3/lib/python3.8/site-packages/")

import scipy
import numpy as np
from cvxopt import matrix, solvers

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from libsvm_util.svmutil import *

In [30]:
BINARY_CLASSIFICATION = True
LAST_DIGIT = 1
BASE_DIR = "../"
train_path = os.path.join(BASE_DIR, "data", "mnist","train.csv")
test_path  = os.path.join(BASE_DIR, "data", "mnist","test.csv")

In [68]:
def load_data(filename, Binary):
    data = np.genfromtxt(filename,delimiter=',')
    data_x = data[:,:784]/255
    data_y = data[:,784]
    data_y = data_y.reshape((data_y.shape[0],1))
    
    if Binary:
        data_x = data_x[(data_y==LAST_DIGIT).reshape(-1) | ( data_y==((LAST_DIGIT+1)%10) ).reshape(-1)]
        data_y = data_y[(data_y==LAST_DIGIT).reshape(-1) | ( data_y==((LAST_DIGIT+1)%10) ).reshape(-1)]
        data_y = -1.0*(data_y==LAST_DIGIT) + 1.0*(data_y==((LAST_DIGIT+1)%10))
        
    return data_x,data_y

In [ ]:
def linear_kernel(X,y):
    mat = np.array(X*y)
    return np.dot(mat, mat.T)

def gaussian_kernel(X,z):
    pass

In [ ]:
class SVM:
    def __init__(self,kernel,C):
        if kernel == "liner":
            self.kernel = linear_kernel
        else:
            self.kernel = gaussian_kernel
            
        self.C = float(C)
    
    def train(self, X_train, Y_train):
        # minimizing function
        if self.kernel == linear_kernel:
            P = matrix(self.kernel(X_train,Y_train))
        else:
            pass
        P = .5 * (P + P.T)  # Just to be on the safe side (ensuring P is symmetric)
        q = matrix(-1.0*np.ones((X_train.shape[0],1)))
        c = 0.0
        
        # Inequalities
        pos = np.ones(X_train.shape[0])
        neg = -np.ones(X_train.shape[0])
        G   = matrix(np.diag(np.concatenate((pos,neg)).reshape(-1)))
        
        zer   = np.zeros((X_train.shape[0]))
        c_val = self.C*np.ones(X_train.shape[0])
        h     = matrix(np.concatenate((zer,c_val)))
        
        # Equality
        A = matrix(Y_train.reshape(-1))
        b = 0.0
        
        solvers.options['show_progress'] = True
        sol = solvers.qp(P, q, G, h, A, b);
        print(sol)

In [69]:
if BINARY_CLASSIFICATION:
    X_train, Y_train = load_data(train_path, BINARY_CLASSIFICATION)
    X_test, Y_test  = load_data(test_path, BINARY_CLASSIFICATION)
    
    
    